# **Récolte des données** 

### Modification à faire :
- ~~Programme pour traiter les données json~~
- Changer le filtrage des catégorie, pas besoin de filtre pandas, ajouter c=id à la fin de l'url
- Expliquer comment définir une bbox correctement
- Compléter la liste des points d'intérêts disponibles
- Retravailler les cellules markdown et les commentaires Python
- Restructurer les paramètres
- **Soucis avec la récolte des données OpenStreetMap, malgré le fait que la bbox soit (normalement) centré sur Brest, on obtient des écoles en Espagne**

### Importation des librairies utiles

In [27]:
import requests
import json
import pandas as pd
import overpy
import geopy.distance as gd

### Création du dictionnaire des Localisation avec leur URL respectives

In [28]:
#Localisation des données
dict_association_localisation = {"AGGLOMÉRATION LE COTENTIN":'https://vigilo.velorution-cherbourg.fr/get_issues.php?',
                                 "AGGLOMÉRATION TROYENNE":'https://vigilo.troyesenselle.fr/get_issues.php?', 
                                 "AIX MARSEILLE PROVENCE METROPOLE":'https://vigilo.velosenville.org/get_issues.php?',
                                 "BASSIN D'ARCACHON":'https://vigilo.velobassin.org/get_issues.php?',
                                 "BEAUVAISIS":'https://vigilo.vellovaque.fr/get_issues.php?',
                                 "BORDEAUX MÉTROPOLE":'https://vigilo.cycles-manivelles.org/get_issues.php?',
                                 "BREST MÉTROPOLE":'https://vigilo.bapav.org/get_issues.php?',
                                 "EVREUX":'https://vigilo.cube27.org/get_issues.php?',
                                 "GRAND MONTPELLIER":'https://vigilo.jesuisundesdeux.org/get_issues.php?',
                                 "LA ROCHELLE":'https://vigilo.-lr.corolo.fr/get_issues.php?',
                                 "LORIENT AGGLO":'https://vigilo.claav.fr/get_issues.php?',
                                 "MONS":'https://vigilo.geckonline.be/get_issues.php?',
                                 "NANCY AGGLOMÉRATION":'https://vigilo.rigelk.eu/get_issues.php?',
                                 "NANTES MÉTROPOLE":'https://vigilo.placeauvelo-nantes.fr/get_issues.php?',
                                 "NORD-ISÈRE (APIE)":'https://vigilo.apie-asso.net/get_issues.php?',
                                 "PARIS SACLAY":'https://api.vigilo-parissaclay.mdb-idf.org/get_issues.php?',
                                 "PAYS DE CHÂTEAUGIRON":'https://pouliquen.net/vigilo/get_issues.php?',
                                 "PÉRIGUEUX AGGLO":'https://vigilo.velorution-perigourdine.fr/get_issues.php?',
                                 "SAINT MARCELLIN VERCORS ISÈRE COMMUNAUTÉ":'https://vigilo.roulavelo.org/get_issues.php?',
                                 "SAINT-BRIEUC ARMOR AGGLOMÉRATION":'https://api-vigilo.velo-utile.fr/get_issues.php?',
                                 "STRASBOURG":'https://www.strasbourgavelo.eu/get_issues.php?',
                                 "SUD OISE":'https://vigilo.au5v.fr//get_issues.php?',
                                 "THAU":'https://prod-sete.vigilo.velocite-montpellier.fr/get_issues.php?',
                                 "VANNES":'https://vigilo.kaz.bzh/get_issues.php?'
                                }

df_localisation = pd.DataFrame(dict_association_localisation.keys(), columns=["Localisation"])
df_localisation.set_index("Localisation", inplace=True)
df_localisation

""
Localisation
AGGLOMÉRATION LE COTENTIN
AGGLOMÉRATION TROYENNE
AIX MARSEILLE PROVENCE METROPOLE
BASSIN D'ARCACHON
BEAUVAISIS
BORDEAUX MÉTROPOLE
BREST MÉTROPOLE
EVREUX
GRAND MONTPELLIER


### Création du dictionnaire avec les différentes catégories de signalements et leur numéro

In [29]:
# URL de l'API pour récupérer la liste des catégories
url_categories = 'https://vigilo-bf7f2.firebaseio.com/categorieslist.json'
resp = requests.get(url_categories)
resp.raise_for_status()

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {cat["catname"]: str(cat["catid"]) for cat in resp.json()}

# Créer un dataframe à partir du dictionnaire
df_categories = pd.DataFrame(categories_dict.items(), columns=["Catégories", "id"])

# Changer l'index et trier par id
df_categories = df_categories.set_index("id")

# Afficher le dataframe
df_categories


,Catégories
id,
8,Absence d'aménagement
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
2,Véhicule ou objet gênant
9,"Accident, chute, incident"
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
10,Vol ou dégradation de vélo


### Liste des POI

In [30]:
# Créer un dictionnaire des points d'intérêt et leur identifiant OSM (ex: pour les parkings à vélo : "amenity:bicycle_parking")

poi_dict = {
    "Écoles": "amenity=school",
    "Hôpitaux": "amenity=hospital",
    "Parkings à vélos": "amenity=bicycle_parking",
    "Stations de vélos en libre-service": "amenity=bicycle_rental",
    "Ateliers de réparation de vélos": "shop=bicycle",
    "Passages piétons": "highway=crossing",
    "Feux piétons": "highway=traffic_signals",
    "Places": "highway=pedestrian",
    "Rampes d'accès pour handicapés": "amenity=wheelchair_ramp",
    "Sièges pour handicapés": "amenity=wheelchair_seating",
    "Parkings pour handicapés": "amenity=parking_space",
    "Accès piétonniers": "footway=sidewalk"
}


# Créer un dataframe à partir du dictionnaire
df_poi = pd.DataFrame(poi_dict.items(), columns=["Points d'intérêt", "Identifiant OSM"])

# Changer l'index
df_poi = df_poi.set_index("Identifiant OSM")

# Afficher le dataframe
df_poi

,Points d'intérêt
Identifiant OSM,
amenity=school,Écoles
amenity=hospital,Hôpitaux
amenity=bicycle_parking,Parkings à vélos
amenity=bicycle_rental,Stations de vélos en libre-service
shop=bicycle,Ateliers de réparation de vélos
highway=crossing,Passages piétons
highway=traffic_signals,Feux piétons
highway=pedestrian,Places
amenity=wheelchair_ramp,Rampes d'accès pour handicapés


### Initialisation des paramètres

In [31]:
# Paramètres de recherche

# Filtre de recherche par localisation, exemple : localisation = "BREST MÉTROPOLE"
location_filter = "BREST MÉTROPOLE"

# Filtre des catégories à afficher, exemple : category_filter = "8" donne la catégorie Absence d'aménagement
category_filter = "Absence d'aménagement"

# Filtre des points d'intérêt à afficher, exemple : poi_filter = "Parking à vélo" donne le point d'intérêt Parking à vélo
poi_name = "Écoles"

# Boîte de recherche pour les points d'intérêt
central_lat = 48.390177
central_lon = -4.496499
delta = 0.03

bbox = [
    central_lat - delta,  # south latitude
    central_lon - delta,  # west longitude
    central_lat + delta,  # north latitude
    central_lon + delta   # east longitude
]

### Création du dataframe des données

In [32]:
def data_collection():
    try:
        resp = requests.get(dict_association_localisation[location_filter])
        resp.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        df = pd.DataFrame(resp.json())
        return df[df['categorie'] == categories_dict[category_filter]] if category_filter else df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None
df = data_collection()

### Aperçu des données récoltées

In [33]:
df.head(10)

,token,coordinates_lat,coordinates_lon,address,comment,explanation,time,status,group,categorie,approved,cityname
156,371F7590,48.38590285501459,-4.5119048196845934,"Rue de la Frégate Laplace, Brest",Proposer un contre-sens cyclable ?,Pourquoi ne pas proposer un contre-sens cyclab...,1686144720,1,0,8,1,Brest
187,32FC146C,48.434539574331865,-4.501100778579713,"Chemin de Lostennic, Brest",Traversée dangereuse,Il est très dangereux de traverser la rue du T...,1685428380,0,0,8,1,Brest
204,EB3F1768,48.38673272551244,-4.470934644341469,"rd point herman melville, Brest",pb d insertion,"l aménagement est super, une grande avancée a...",1684163460,0,0,8,1,Brest
242,CE8F870B,48.38747467145252,-4.4770628213882455,"Rue Poullic Al Lor, Brest",Absence de trottoir,,1681050960,0,0,8,1,Brest
262,F4D753AA,48.4022149826915,-4.492029001716181,"Rue Marcel Sembat, Brest",Trottoir impraticable depuis des mois.,Trottoir défoncé par les camions livrant des m...,1675073700,0,0,8,1,Brest
299,40FBF5B1,48.40211174589075,-4.48873445391655,"Rue Auguste Kervern, Brest",Aménagements cyclables prévus ?,Aménagements cyclables de prévus ? \nRappelons...,1665509400,3,0,8,1,Brest
302,BF380A40,48.39031492242171,-4.491557478904724,"Rue de Lyon, Brest.",Pas de bande cyclable.,rue assez large ne disposant pas d'aménagement...,1665249660,0,0,8,1,Brest.
303,FAA931F6,48.403994371926466,-4.497075453400612,"Professeur Langevin, Brest",absence de bande cyclable.,Il n'y a pas d'aménagements cyclables. Le sens...,1665132240,0,0,8,1,Brest
304,6A2F06A8,48.404106330111496,-4.4963472336530685,"Joseph le Borgne, Brest",Aménagements prévus ?,Rue actuellement en travaux. L'article de Loi ...,1665132240,0,0,8,1,Brest
333,DFE92076,48.40029518976901,-4.480251954523882,"Rue Paul Masson, Brest",Manque un passage piéton,,1662741480,0,0,8,1,Brest


In [35]:
# Trier les données en fonction de leur coordonnées GPS, si elles sont dans la boîte de recherche autour du point d'intérêt
def fetch_poi_data(bbox, poi_name, poi_dict):
    # Fetch the tag for the poi_name from the poi_dict
    poi_tag = poi_dict.get(poi_name)
    if poi_tag is None:
        raise ValueError(f"Le point d'intérêt '{poi_name}' n'est pas reconnu.")
    
    # Format the bounding box and tags correctly for the Overpass query
    formatted_bbox = f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}"
    formatted_poi_tag = poi_tag.replace("=", '="') + '"'

    # Define the Overpass API URL
    url = "http://overpass-api.de/api/interpreter"

    # Construct the query using the formatted bbox and poi_tag
    query = f"""
    [out:json];
    (
        node[{formatted_poi_tag}](bbox:{formatted_bbox});
        way[{formatted_poi_tag}](bbox:{formatted_bbox});
        rel[{formatted_poi_tag}](bbox:{formatted_bbox});
    );
    out center;
    """

    # Send the request to the Overpass API
    response = requests.get(url, params={'data': query})
    response.raise_for_status()  # Raises an HTTPError for bad responses
    return response.json()

def process_poi_data(json_data):
    # Initialize a dictionary to hold all POIs
    poi_dict = {}
    
    # Go through each element in the JSON data
    for element in json_data['elements']:
        # Check if the element is a node and has the required 'tags'
        if element['type'] == 'node' and 'tags' in element and 'amenity' in element['tags']:
            # Use the node ID as the key for the main dictionary
            poi_id = element['id']
            # Create a dictionary for this particular POI
            poi_info = {
                'name': element['tags'].get('name', 'Unknown'),
                'latitude': element['lat'],
                'longitude': element['lon'],
                'address': element['tags'].get('addr:street', 'N/A') + ' ' + element['tags'].get('addr:housenumber', ''),
                'postcode': element['tags'].get('addr:postcode', 'N/A'),
                'city': element['tags'].get('addr:city', 'N/A'),
                'country': element['tags'].get('addr:country', 'N/A'),
                'type': element['tags'].get('school:FR', element['tags'].get('amenity', 'N/A'))
            }
            # Add this POI's information to the main dictionary
            poi_dict[poi_id] = poi_info
    
    return poi_dict

pois = process_poi_data(fetch_poi_data(bbox, poi_name, poi_dict))
df_poi = pd.DataFrame.from_dict(pois, orient='index')
df_poi.head(10)

,name,latitude,longitude,address,postcode,city,country,type
266168377,Centro Privado de Educación Primaria y Secunda...,40.312532,-4.514104,N/A,28649,N/A,N/A,school
266169847,Colegio de Educación Infantil y Primaria Carlo...,40.310629,-4.492775,Calle Antonio Machado 48,28649,N/A,N/A,school
727458152,Lycée général privé Sainte-Anne,48.396623,-4.490729,N/A,N/A,N/A,N/A,lycée
727458154,Lycée polyvalent et professionnel Jules Lesven,48.406575,-4.479869,N/A,N/A,N/A,N/A,lycée
727458156,Lycée Naval,48.369127,-4.525406,N/A,N/A,N/A,N/A,lycée
727458160,Lycée La Pérouse-Kérichen,48.405368,-4.482298,Rue Prince de Joinville,N/A,N/A,N/A,lycée
727523418,École primaire privée de la Providence,48.396179,-4.482862,N/A,N/A,N/A,N/A,primaire
727523612,École primaire publique Kerhallet,48.402529,-4.516355,N/A,N/A,N/A,N/A,primaire
727544532,Collège privé Sainte-Anne,48.395836,-4.491997,N/A,N/A,N/A,N/A,collège
727544535,Collège de la Croix Rouge,48.408144,-4.489686,N/A,N/A,N/A,N/A,collège
